# Classification Exercise - Solutions

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [3]:
import pandas as pd

In [4]:
census = pd.read_csv("census_data.csv")

In [5]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [6]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [7]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [8]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [9]:
# Cool Alternative
# lambda label:int(label==' <=50k')

# census['income_bracket'].apply(lambda label: int(label==' <=50K'))

### Perform a Train Test Split on the Data

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x_data = census.drop('income_bracket',axis=1)
y_labels = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [12]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

** Import Tensorflow **

In [13]:
import tensorflow as tf

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [14]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [15]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

** Put all these variables into a single list with the variable name feat_cols **

In [16]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [17]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [18]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_train_distribute': None, '_service': None, '_keep_checkpoint_max': 5, '_device_fn': None, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_experimental_max_worker_delay_secs': None, '_is_chief': True, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f317b025470>, '_task_type': 'worker', '_model_dir': '/tmp/tmpmwsl93o2', '_evaluation_master': '', '_num_worker_replicas': 1, '_protocol': None, '_eval_distribute': None, '_master': '', '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_global_id_in_cluster': 0, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_summary_steps': 100, '_experimental_distribute': None, '_log_step_count_steps': 100}


** Train your model on the data, for at least 5000 steps. **

In [19]:
model.train(input_fn=input_func,steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpmwsl93o2/model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 264.763
INFO:tensorflow:loss = 780.58386, step = 101 (0.378 sec)
INFO

INFO:tensorflow:global_step/sec: 506.7
INFO:tensorflow:loss = 53.816364, step = 3501 (0.197 sec)
INFO:tensorflow:global_step/sec: 510.657
INFO:tensorflow:loss = 86.227684, step = 3601 (0.196 sec)
INFO:tensorflow:global_step/sec: 519.84
INFO:tensorflow:loss = 23.550797, step = 3701 (0.192 sec)
INFO:tensorflow:global_step/sec: 524.357
INFO:tensorflow:loss = 37.318867, step = 3801 (0.191 sec)
INFO:tensorflow:global_step/sec: 498.973
INFO:tensorflow:loss = 243.39116, step = 3901 (0.200 sec)
INFO:tensorflow:global_step/sec: 500.002
INFO:tensorflow:loss = 45.183243, step = 4001 (0.200 sec)
INFO:tensorflow:global_step/sec: 503.57
INFO:tensorflow:loss = 49.80725, step = 4101 (0.199 sec)
INFO:tensorflow:global_step/sec: 526.808
INFO:tensorflow:loss = 66.72059, step = 4201 (0.190 sec)
INFO:tensorflow:global_step/sec: 499.684
INFO:tensorflow:loss = 41.034317, step = 4301 (0.200 sec)
INFO:tensorflow:global_step/sec: 500.721
INFO:tensorflow:loss = 43.514423, step = 4401 (0.199 sec)
INFO:tensorflow:

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [20]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [21]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmpmwsl93o2/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Each item in your list will look like this: **

In [22]:
predictions[0]

{'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'logistic': array([0.27505398], dtype=float32),
 'logits': array([-0.9691299], dtype=float32),
 'probabilities': array([0.724946  , 0.27505395], dtype=float32)}

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [28]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [30]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [31]:
from sklearn.metrics import classification_report

In [32]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       0.88      0.92      0.90      7436
          1       0.70      0.59      0.64      2333

avg / total       0.84      0.84      0.84      9769



# Great Job!